In [1]:
import warnings
warnings.filterwarnings('ignore')
import sys
print(sys.executable) #check jupyter kernel
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

/CCAS/home/ruimiao/mylustre/miniconda3/envs/proxITR/bin/python


In [2]:
from data.simu_setting import prox_data
from src.proxITR import proxITR
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler as Scaler
from sklearn.kernel_approximation import Nystroem

### Value Function

In [3]:
def value_fun(d,A,Y,X,U):
    A = A.to_numpy().reshape(-1)
    Y = Y.to_numpy().reshape(-1)
    U = U.to_numpy().reshape(-1)
    X = X.to_numpy()
    return np.mean((np.abs((np.sign(A-0.5)-d))<0.5)*Y*(1. + np.exp(np.sign(A-0.5)*(0.09375+X @ [0.1875, 0.1875] -0.25*U))))

## Examples for Different Simulation Settings

### Scenario L1

In [4]:
# Setting
linearity    = 'linear'
Xonly        = 'XW'

In [5]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.256284945993542

In [6]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [7]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.8950567 5.8950567 5.8982506 6.0638638 6.214181  6.2500772 6.2598104
 6.265973  6.268476 ]
rho_best: 0.015625
d2_XW
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.88093571 5.88093571 5.88093571 5.88093571 5.88093571 5.92091311
 6.14882555 6.11784587 6.21730574]
rho_best: 0.015625
d_DR
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.85533191 5.85533191 5.85533191 5.92025939 6.00078612 6.11854926
 6.17626463 6.27231556 6.28126817]
rho_best: 0.015625
d1_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.8950567 5.8950567 5.8982506 6.0592422 6.2125154 6.2487807 6.260869
 6.267933  6.268437 ]
rho_best: 0.015625
d2_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.88093571 5.88093571 

In [8]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

,method,value
0,d1_XZ,6.141538
1,d2_XW,6.190247
2,d4,6.141538
3,d_DR,6.192596
4,d1_X,6.166584
5,d2_X,6.193099


### Confidence interval of $V(d_3^*)$

In [9]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(6.038944169219989, 6.655130044635315)

In [10]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.250627821379017

### Scenario L2

In [11]:
# Setting
linearity    = 'linear'
Xonly        = 'X'

In [12]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.20088632242159

In [13]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [14]:
# Learn ITRs
rhos = np.power(2.,-np.arange(-2,7))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                                learning_rate= 0.1, n_epoch=2000, opt='LBFGS', verbose=True)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])

d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])

d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos, learning_rate=1e-2)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.6966805 5.6966805 6.0558143 6.2464986 6.27661   6.289986  6.2930045
 6.295498  6.2933207]
rho_best: 0.03125
d2_XW
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.65959646 5.65959646 5.65959646 5.65959646 5.73920416 6.30154632
 6.35389729 6.35663714 6.31120067]
rho_best: 0.03125
d_DR
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.63420332 5.63420332 5.68668403 6.05202427 6.23720298 6.23442043
 6.23839025 6.18523893 6.16570121]
rho_best: 0.0625
d1_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.6966805 5.6966805 6.0571995 6.2471647 6.2707496 6.2821198 6.2907686
 6.291161  6.295474 ]
rho_best: 0.015625
d2_X
rho:  [4.       2.       1.       0.5      0.25     0.125    0.0625   0.03125
 0.015625]
value:  [5.65959646 5.65959646 5.6

In [15]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

,method,value
0,d1_XZ,6.141174
1,d2_XW,6.178980
2,d4,6.178980
3,d_DR,6.176762
4,d1_X,6.169085
5,d2_X,6.186525


### Confidence interval of $V(d_3^*)$

In [16]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(5.994741351394222, 6.62843162090164)

In [17]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

6.20088632242159

### Scenario N1

In [18]:
# Setting
linearity    = 'nonlinear'
Xonly        = 'XW'

In [19]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.876093221934032

In [20]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [21]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=True)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.3217807 4.3391037 4.3159766 4.341706  4.299367  4.3182373 4.3171363
 4.3207674 4.327513 ]
rho_best: 0.25
d2_XW
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.28515094 4.36394815 4.29866017 4.27876023 4.31009891 4.29415606
 4.35037879 4.3395206  4.27715377]
rho_best: 1.0
d_DR
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.25853748 4.25794429 4.24830367 4.25164098 4.26545232 4.26577962
 4.25913953 4.24787517 4.26064287]
rho_best: 0.0625
d1_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.187278  4.2072067 4.225553  4.198153  4.1659217 4.176115  4.193451
 4.188915  4.1662683]
rho_best: 0.5
d2_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value: 

In [22]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

,method,value
0,d1_XZ,4.306270
1,d2_XW,4.413050
2,d4,4.413050
3,d_DR,4.212884
4,d1_X,4.207721
5,d2_X,4.193253


### Confidence interval of $V(d_3^*)$

In [23]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(4.120200116546298, 4.450948111696376)

In [24]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.24388460433744

### Scenario N2

In [25]:
# Setting
linearity    = 'nonlinear'
Xonly        = 'X'

In [26]:
# Generate Test Data
test = prox_data(500000, add_noise=False).gen_Y(linearity=linearity, Xonly = Xonly)
value_fun(test[["GOR"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.594857933479509

In [27]:
# Generate Training Data
samp_size = 2000
train = prox_data(samp_size=samp_size, add_noise=True).gen_Y(linearity=linearity, Xonly = Xonly)

In [28]:
rhos = np.power(2.,-np.arange(-1,8))
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
learning_rate=0.1, n_epoch=2000, batch_size=300, opt='SGD', verbose=True)
d1_XZ = proxy.fit_d1_XZ_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_XZ = d1_XZ(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","Z"]])
d2_XW = proxy.fit_d2_XW_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_XW = d2_XW(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9","W"]])
d_DR  = proxy.fit_d_DR_cv(gamma_f='auto', n_gamma_hs=10, 
        h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d_DR  = d_DR(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d1_X  = proxy.fit_d1_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d1_X  = d1_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])
d2_X  = proxy.fit_d2_X_cv(gamma_f='auto', n_gamma_hs=10, 
        alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
        linearity=linearity, n_components=int(2*np.sqrt(samp_size)), rhos = rhos)
p_d2_X  = d2_X(test[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]])

d1_XZ
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.4775558 4.484598  4.486319  4.4849863 4.488392  4.488771  4.4868493
 4.4868455 4.487235 ]
rho_best: 0.0625
d2_XW
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.3900914  4.44973586 4.38763282 4.30227377 4.33981581 4.3734924
 4.45231745 4.35230872 4.33392341]
rho_best: 0.03125
d_DR
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.40526537 4.48159457 4.47938369 4.48930593 4.48476484 4.51620178
 4.4994138  4.45630692 4.49719566]
rho_best: 0.0625
d1_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
value:  [4.4709473 4.479787  4.4825077 4.4825945 4.4820294 4.4813094 4.481656
 4.4805665 4.4819627]
rho_best: 0.25
d2_X
rho:  [2.        1.        0.5       0.25      0.125     0.0625    0.03125
 0.015625  0.0078125]
v

In [29]:
# Calculate Value
v_d1_XZ = value_fun(p_d1_XZ,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_XW = value_fun(p_d2_XW,test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d_DR  = value_fun(p_d_DR, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d1_X  = value_fun(p_d1_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
v_d2_X  = value_fun(p_d2_X, test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])
if proxy.cv_d1_XZ > proxy.cv_d2_XW:
    v_d4 = v_d1_XZ
else:
    v_d4 = v_d2_XW
# Table of values
pd.DataFrame.from_dict({'method':['d1_XZ','d2_XW','d4','d_DR','d1_X','d2_X'],
                        'value':[v_d1_XZ,v_d2_XW,v_d4,v_d_DR,v_d1_X,v_d2_X]})

,method,value
0,d1_XZ,4.483465
1,d2_XW,4.494700
2,d4,4.483465
3,d_DR,4.530027
4,d1_X,4.538844
5,d2_X,4.501892


### Confidence interval of $V(d_3^*)$

In [30]:
proxy = proxITR(A=train[["A"]], X=train[["X0","X1","X2","X3","X4","X5","X6","X7","X8","X9"]], Z=train[["Z"]], W=train[["W"]], Y=train[["Y"]], 
                god=True, d_X_opt = train[["GOR_X"]])
CI = proxy.eif_CI(gamma_f='auto', n_gamma_hs=10, n_alphas=20,
            h_alpha_scales=[c for c in np.geomspace(0.5, 1e5, 20)],
            q_alpha_scales=[c for c in np.geomspace(0.1, 1e3, 20)],
            n_components=int(2*np.sqrt(samp_size)))
CI

(4.285745655923722, 4.665195771756086)

In [31]:
# V(d_3^*)
value_fun(test[["GOR_X"]].to_numpy(int).reshape(-1), test[["A"]],test[["Y"]],test[["X0","X1"]],test[["U"]])

4.594857933479509